In [92]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
import math

In [125]:
#Reading in 2016 data we cleaned
df2016 = pd.read_csv('data/exported_data.csv')


In [126]:
# Reading in 2020 data we collected 

# Population broken down by age
state13 = pd.read_csv('data/cc-est2019-agesex-13.csv')
state19 = pd.read_csv('data/cc-est2019-agesex-19.csv')
state12 = pd.read_csv('data/cc-est2019-agesex-12.csv')
state27 = pd.read_csv('data/cc-est2019-agesex-27.csv')
state32 = pd.read_csv('data/cc-est2019-agesex-32.csv')
state37 = pd.read_csv('data/cc-est2019-agesex-37.csv')
state39 = pd.read_csv('data/cc-est2019-agesex-39.csv')
state42 = pd.read_csv('data/cc-est2019-agesex-42.csv')
state48 = pd.read_csv('data/cc-est2019-agesex-48.csv')
state55 = pd.read_csv('data/cc-est2019-agesex-55.csv')
state26 = pd.read_csv('data/cc-est2019-agesex-26.csv')
state4 = pd.read_csv('data/cc-est2019-agesex-04.csv')
df_states = state13.append(state19).append(state12).append(state27).append(state32).append(state37).append(state39).append(state42).append(state48).append(state55).append(state26).append(state4)

df_states = df_states[df_states['YEAR'] == 12] #12 represents 2019 (as close to 2020 as we could find)




In [127]:
df_states.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
11,50,13,1,Georgia,Appling County,12,18386,9157,9229,1177,608,569,2359,1186,1173,977,492,485,1520,848,672,14347,7115,7232,13873,6871,7002,6592,3430,3162,4366,2221,2145,4730,2324,2406,3257,1478,1779,1177,608,569,1306,655,651,1324,662,662,1124,598,526,1102,611,491,1127,620,507,1017,491,526,1090,555,535,1132,555,577,1111,537,574,1128,578,550,1241,624,617,1250,585,665,1006,482,524,929,465,464,605,272,333,398,150,248,319,109,210,39.7,37.9,41.2
23,50,13,3,Georgia,Atkinson County,12,8165,4146,4019,573,300,273,1069,550,519,498,259,239,751,373,378,6280,3175,3105,6025,3037,2988,3164,1620,1544,2035,1048,987,2056,1043,1013,1183,573,610,573,300,273,595,307,288,594,303,291,584,300,284,545,272,273,547,280,267,498,269,229,462,228,234,528,271,257,512,265,247,534,297,237,505,241,264,505,240,265,392,192,200,325,161,164,228,105,123,148,71,77,90,44,46,36.6,35.9,37.4
35,50,13,5,Georgia,Bacon County,12,11164,5455,5709,766,381,385,1424,719,705,643,313,330,940,500,440,8651,4204,4447,8331,4042,4289,4219,2125,2094,2788,1387,1401,2725,1340,1385,1878,815,1063,766,381,385,774,390,384,802,404,398,753,381,372,678,357,321,684,348,336,679,346,333,631,290,341,794,403,391,717,372,345,706,350,356,629,313,316,673,305,368,671,318,353,479,228,251,323,131,192,243,90,153,162,48,114,38.6,37.2,39.8
47,50,13,7,Georgia,Baker County,12,3038,1494,1544,175,97,78,281,139,142,135,58,77,227,109,118,2513,1228,1285,2447,1200,1247,947,469,478,618,317,301,883,436,447,719,338,381,175,97,78,138,75,63,176,79,97,162,74,88,167,78,89,169,93,76,156,80,76,151,67,84,142,77,65,196,98,98,190,90,100,251,120,131,246,128,118,238,118,120,178,80,98,131,63,68,69,29,40,103,48,55,47.3,46.8,47.9
59,50,13,9,Georgia,Baldwin County,12,44890,22737,22153,2113,1116,997,4262,2158,2104,1913,991,922,8380,4086,4294,37528,18950,18578,36602,18472,18130,19737,10098,9639,9947,5283,4664,10877,5662,5215,7398,3441,3957,2113,1116,997,2278,1130,1148,2487,1290,1197,4360,2049,2311,5430,2766,2664,2846,1517,1329,2420,1286,1134,2452,1284,1168,2229,1196,1033,2458,1320,1138,2631,1397,1234,2925,1490,1435,2863,1455,1408,2452,1203,1249,2008,916,1092,1333,617,716,863,384,479,742,321,421,36.1,35.9,36.3


In [128]:
# Getting the columns we want
df_states = df_states[['STATE', 'COUNTY', 'STNAME', 'CTYNAME','AGE18PLUS_TOT','AGE18PLUS_MALE','AGE18PLUS_FEM', 'AGE1824_TOT', 'AGE1824_MALE','AGE1824_FEM', 
                       'AGE2544_TOT','AGE2544_MALE','AGE2544_FEM', 'AGE4564_TOT','AGE4564_MALE', 'AGE4564_FEM',
                      'AGE65PLUS_TOT', 'AGE65PLUS_MALE', 'AGE65PLUS_FEM']]

In [129]:
# Dividing by total value to get a percentage 
df_states['1824tot_per'] = df_states['AGE1824_TOT']/df_states['AGE18PLUS_TOT']
df_states['1824M_per'] = df_states['AGE1824_MALE']/df_states['AGE18PLUS_TOT']
df_states['1824F_per'] = df_states['AGE1824_FEM']/df_states['AGE18PLUS_TOT']
df_states['2544tot_per'] = df_states['AGE2544_TOT']/df_states['AGE18PLUS_TOT']
df_states['2544M_per'] = df_states['AGE2544_MALE']/df_states['AGE18PLUS_TOT']
df_states['2544F_per'] = df_states['AGE2544_FEM']/df_states['AGE18PLUS_TOT']
df_states['4564tot_per'] = df_states['AGE4564_TOT']/df_states['AGE18PLUS_TOT']
df_states['4564M_per'] = df_states['AGE4564_MALE']/df_states['AGE18PLUS_TOT']
df_states['4564F_per'] = df_states['AGE4564_FEM']/df_states['AGE18PLUS_TOT']
df_states['65tot_per'] = df_states['AGE65PLUS_TOT']/df_states['AGE18PLUS_TOT']
df_states['65M_per'] = df_states['AGE65PLUS_MALE']/df_states['AGE18PLUS_TOT']
df_states['65F_per'] = df_states['AGE65PLUS_FEM']/df_states['AGE18PLUS_TOT']

In [130]:
df_states.head()

,STATE,COUNTY,STNAME,CTYNAME,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,1824tot_per,1824M_per,1824F_per,2544tot_per,2544M_per,2544F_per,4564tot_per,4564M_per,4564F_per,65tot_per,65M_per,65F_per
11,13,1,Georgia,Appling County,13873,6871,7002,1520,848,672,4366,2221,2145,4730,2324,2406,3257,1478,1779,0.109565,0.061126,0.048439,0.314712,0.160095,0.154617,0.340950,0.167520,0.173430,0.234773,0.106538,0.128235
23,13,3,Georgia,Atkinson County,6025,3037,2988,751,373,378,2035,1048,987,2056,1043,1013,1183,573,610,0.124647,0.061909,0.062739,0.337759,0.173942,0.163817,0.341245,0.173112,0.168133,0.196349,0.095104,0.101245
35,13,5,Georgia,Bacon County,8331,4042,4289,940,500,440,2788,1387,1401,2725,1340,1385,1878,815,1063,0.112832,0.060017,0.052815,0.334654,0.166487,0.168167,0.327092,0.160845,0.166247,0.225423,0.097827,0.127596
47,13,7,Georgia,Baker County,2447,1200,1247,227,109,118,618,317,301,883,436,447,719,338,381,0.092767,0.044544,0.048222,0.252554,0.129546,0.123008,0.360850,0.178177,0.182673,0.293829,0.138128,0.155701
59,13,9,Georgia,Baldwin County,36602,18472,18130,8380,4086,4294,9947,5283,4664,10877,5662,5215,7398,3441,3957,0.228949,0.111633,0.117316,0.271761,0.144336,0.127425,0.297170,0.154691,0.142479,0.202120,0.094011,0.108109


In [131]:
df_states = df_states.reset_index()

In [156]:
# 2019 American Community Survey data about education level
acs_edu = pd.read_csv('data/acs_data_education.csv', header = 1)

In [157]:
acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree","Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Margin of Error!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT

In [158]:
# 2019 American Community Survey data about racial makeup
acs_race = pd.read_csv('data/acs_data_race.csv', header = 1)

In [159]:
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Margin of Error!!Total:,Estimate!!Total:!!White alone,Margin of Error!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Margin of Error!!Total:!!Black or African American alone,Estimate!!Total:!!American Indian and Alaska Native alone,Margin of Error!!Total:!!American Indian and Alaska Native alone,Estimate!!Total:!!Asian alone,Margin of Error!!Total:!!Asian alone,Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total:!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total:!!Some other race alone,Margin of Error!!Total:!!Some other race alone,Estimate!!Total:!!Two or more races:,Margin of Error!!Total:!!Two or more races:,Estimate!!Total:!!Two or more races:!!Two races including Some other race,Margin of Error!!Total:!!Two or more races:!!Two races including Some other race,"Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races","Margin of Error!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races"
0,0500000US04001,"Apache County, Arizona",71887.0,*****,15109.0,899.0,721.0,434.0,53480.0,1032.0,160.0,168.0,15.0,27.0,1127.0,595.0,1275.0,743.0,637.0,509.0,638.0,514.0
1,0500000US04003,"Cochise County, Arizona",125922.0,*****,108177.0,1768.0,6163.0,1237.0,1006.0,911.0,2795.0,674.0,129.0,218.0,2141.0,1071.0,5511.0,1259.0,448.0,334.0,5063.0,1164.0
2,0500000US04005,"Coconino County, Arizona",143476.0,*****,91649.0,1736.0,1364.0,461.0,37187.0,1082.0,2730.0,481.0,419.0,440.0,3897.0,1334.0,6230.0,1766.0,253.0,305.0,5977.0,1750.0
3,0500000US04013,"Maricopa County, Arizona",4485414.0,*****,3547155.0,17870.0,266128.0,7213.0,93358.0,6179.0,192301.0,5275.0,10045.0,1308.0,213953.0,16250.0,162474.0,9681.0,31995.0,5459.0,130479.0,8908.0
4,0500000US04015,"Mohave County, Arizona",212181.0,*****,188756.0,3072.0,1496.0,845.0,4358.0,1173.0,1923.0,474.0,138.0,225.0,7608.0,3077.0,7902.0,1920.0,1169.0,605.0,6733.0,1827.0


In [160]:
# Dropping columns we don't need in both of them

cols_edu = []
for i in range(3,56):
    if i%2!=0:
        cols_edu.append(i)
        
cols_edu_rest = list(range(56,770))

acs_edu = acs_edu.drop(acs_edu.columns[cols_edu_rest], axis = 1)
acs_edu = acs_edu.drop(acs_edu.columns[cols_edu], axis = 1)
acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!Bachelor's degree or higher
0,0500000US04001,"Apache County, Arizona",6532,1054,2127,3187,164,46255,4149,4297,14470,13772,4095,2837,2635,37809,5472,9316,8025,860,7153,6682,801,18183,15457,2125,11603,7645,1686
1,0500000US04003,"Cochise County, Arizona",10486,2191,3103,5114,78,88433,4006,6043,23731,22329,10223,13332,8769,78384,22101,15467,13438,3639,14188,12235,3504,29839,26707,6641,28939,26004,8317
2,0500000US04005,"Coconino County, Arizona",30789,1290,8590,18963,1946,84682,1910,3606,17279,21288,7088,18637,14874,79166,33511,19997,19184,7735,15283,14223,6963,30816,29051,10833,18586,16708,7980
3,0500000US04013,"Maricopa County, Arizona",415899,58736,149802,155679,51682,3017076,157753,200414,671626,710797,264954,641198,370334,2658909,1011532,658682,592116,215671,583814,501914,203036,1078113,941895,357351,696467,622984,235474
4,0500000US04015,"Mohave County, Arizona",13365,2782,6209,3876,498,163211,4079,17157,55224,50138,14602,13958,8053,141975,22011,20109,16498,1469,19281,17378,3652,57994,51636,7972,65827,56463,8918


In [161]:
acs_edu['Less than High School'] = (acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade'])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'] + 
    acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years'])
acs_edu['At Least High School'] = (acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency)'] + 
    acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency)"])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])

acs_edu["At Least Bachelor's Degree"] = (acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"]+
                                         acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree"])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])
acs_edu['Graduate Degree'] = (acs_edu["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"]+
                             acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree'])/(acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years']+
                                                                                                                                     acs_edu['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over'])


acs_edu.head()

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate (includes equivalency),"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Some college, no degree",Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Graduate or professional degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 to 34 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 35 to 44 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 45 to 64 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!High school graduate or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 65 years and over!!Bachelor's degree or higher,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree
0,0500000US04001,"Apache County, Arizona",6532,1054,2127,3187,164,46255,4149,4297,14470,13772,4095,2837,2635,37809,5472,9316,8025,860,7153,6682,801,18183,15457,2125,11603,7645,1686,0.179969,0.314415,0.056851,0.053024
1,0500000US04003,"Cochise County, Arizona",10486,2191,3103,5114,78,88433,4006,6043,23731,22329,10223,13332,8769,78384,22101,15467,13438,3639,14188,12235,3504,29839,26707,6641,28939,26004,8317,0.123738,0.271272,0.135565,0.089437
2,0500000US04005,"Coconino County, Arizona",30789,1290,8590,18963,1946,84682,1910,3606,17279,21288,7088,18637,14874,79166,33511,19997,19184,7735,15283,14223,6963,30816,29051,10833,18586,16708,7980,0.058941,0.224030,0.178253,0.145664
3,0500000US04013,"Maricopa County, Arizona",415899,58736,149802,155679,51682,3017076,157753,200414,671626,710797,264954,641198,370334,2658909,1011532,658682,592116,215671,583814,501914,203036,1078113,941895,357351,696467,622984,235474,0.121441,0.239276,0.201831,0.122930
4,0500000US04015,"Mohave County, Arizona",13365,2782,6209,3876,498,163211,4079,17157,55224,50138,14602,13958,8053,141975,22011,20109,16498,1469,19281,17378,3652,57994,51636,7972,65827,56463,8918,0.136021,0.347913,0.081868,0.048427


In [162]:
cols_edu = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
acs_edu = acs_edu.drop(acs_edu.columns[cols_edu],axis = 1)
acs_edu.head()

,id,Geographic Area Name,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree
0,0500000US04001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024
1,0500000US04003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437
2,0500000US04005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664
3,0500000US04013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930
4,0500000US04015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427


In [163]:
cols_race = [3,5,7,8,9,11,12,13,14,15,16,17,18,19,20,21]
acs_race = acs_race.drop(acs_race.columns[cols_race], axis = 1)
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Estimate!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Estimate!!Total:!!Asian alone
0,0500000US04001,"Apache County, Arizona",71887.0,15109.0,721.0,160.0
1,0500000US04003,"Cochise County, Arizona",125922.0,108177.0,6163.0,2795.0
2,0500000US04005,"Coconino County, Arizona",143476.0,91649.0,1364.0,2730.0
3,0500000US04013,"Maricopa County, Arizona",4485414.0,3547155.0,266128.0,192301.0
4,0500000US04015,"Mohave County, Arizona",212181.0,188756.0,1496.0,1923.0


In [164]:
#Converting them to percentages 

acs_race['White (Not Latino) Population'] = acs_race['Estimate!!Total:!!White alone']/acs_race['Estimate!!Total:'] 
acs_race['African American Population'] = acs_race['Estimate!!Total:!!Black or African American alone']/acs_race['Estimate!!Total:']
acs_race['Asian American Population'] = acs_race['Estimate!!Total:!!Asian alone']/acs_race['Estimate!!Total:']

acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,Estimate!!Total:!!White alone,Estimate!!Total:!!Black or African American alone,Estimate!!Total:!!Asian alone,White (Not Latino) Population,African American Population,Asian American Population
0,0500000US04001,"Apache County, Arizona",71887.0,15109.0,721.0,160.0,0.210177,0.010030,0.002226
1,0500000US04003,"Cochise County, Arizona",125922.0,108177.0,6163.0,2795.0,0.859079,0.048943,0.022196
2,0500000US04005,"Coconino County, Arizona",143476.0,91649.0,1364.0,2730.0,0.638776,0.009507,0.019028
3,0500000US04013,"Maricopa County, Arizona",4485414.0,3547155.0,266128.0,192301.0,0.790820,0.059332,0.042873
4,0500000US04015,"Mohave County, Arizona",212181.0,188756.0,1496.0,1923.0,0.889599,0.007051,0.009063


In [165]:
# Dropping other columns we don't need now
acs_race = acs_race.drop(acs_race.columns[[3,4,5]], axis = 1)
acs_race.head()

,id,Geographic Area Name,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population
0,0500000US04001,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226
1,0500000US04003,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196
2,0500000US04005,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028
3,0500000US04013,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873
4,0500000US04015,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063


In [166]:
#Merge these two

df2020 = pd.merge(left = acs_edu, right = acs_race, on = 'id')
df2020.head()

,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Geographic Area Name_y,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population
0,0500000US04001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,"Apache County, Arizona",71887.0,0.210177,0.010030,0.002226
1,0500000US04003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,"Cochise County, Arizona",125922.0,0.859079,0.048943,0.022196
2,0500000US04005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,"Coconino County, Arizona",143476.0,0.638776,0.009507,0.019028
3,0500000US04013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,"Maricopa County, Arizona",4485414.0,0.790820,0.059332,0.042873
4,0500000US04015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427,"Mohave County, Arizona",212181.0,0.889599,0.007051,0.009063


In [167]:
# Drop repeated column from joining 
df2020 = df2020.drop(['Geographic Area Name_y'],axis = 1)

In [168]:
# Reading in unemployment data
unemployment = pd.read_csv('data/unemployment2020.csv', header = 4)
unemployment.head()

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CN0100100000000,1.0,1.0,"Autauga County, AL",Sep-19,"26,010","25,391",619,2.4
2,CN0100300000000,1.0,3.0,"Baldwin County, AL",Sep-19,"96,754","94,510","2,244",2.3
3,CN0100500000000,1.0,5.0,"Barbour County, AL",Sep-19,"8,656","8,376",280,3.2
4,CN0100700000000,1.0,7.0,"Bibb County, AL",Sep-19,"8,655","8,430",225,2.6


In [169]:
unemployment.tail(20)

,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
45050,CN7212100000000,72.0,121.0,"Sabana Grande Municipio, PR",Oct-20 p,"5,624","5,151",473,8.4
45051,CN7212300000000,72.0,123.0,"Salinas Municipio, PR",Oct-20 p,"6,431","5,745",686,10.7
45052,CN7212500000000,72.0,125.0,"San German Municipio, PR",Oct-20 p,"9,034","8,276",758,8.4
45053,CN7212700000000,72.0,127.0,"San Juan Municipio, PR",Oct-20 p,"128,360","117,918","10,442",8.1
45054,CN7212900000000,72.0,129.0,"San Lorenzo Municipio, PR",Oct-20 p,"10,771","9,910",861,8.0
45055,CN7213100000000,72.0,131.0,"San Sebastian Municipio, PR",Oct-20 p,"9,510","8,539",971,10.2
45056,CN7213300000000,72.0,133.0,"Santa Isabel Municipio, PR",Oct-20 p,"8,208","7,593",615,7.5
45057,CN7213500000000,72.0,135.0,"Toa Alta Municipio, PR",Oct-20 p,"25,611","23,731","1,880",7.3
45058,CN7213700000000,72.0,137.0,"Toa Baja Municipio, PR",Oct-20 p,"27,770","25,551","2,219",8.0
45059,CN7213900000000,72.0,139.0,"Trujillo Alto Municipio, PR",Oct-20 p,"25,534","23,915","1,619",6.3


In [170]:
unemployment.Period.unique()

array([nan, 'Sep-19', 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20',
       'Mar-20', 'Apr-20', 'May-20', 'Jun-20', 'Jul-20', 'Aug-20',
       'Sep-20', 'Oct-20 p'], dtype=object)

In [171]:
unemployment.dtypes

LAUS Code                          object
Code                              float64
Code.1                            float64
County Name/State Abbreviation     object
Period                             object
Force                              object
Employed                           object
Unemployed                         object
(%)                                object
dtype: object

In [172]:
# Converting code and code.1 to ints
unemployment = unemployment.fillna(0)
unemployment.Code = unemployment.Code.astype(int)
unemployment['Code.1'] = unemployment['Code.1'].astype(int)

unemployment.dtypes

LAUS Code                         object
Code                               int64
Code.1                             int64
County Name/State Abbreviation    object
Period                            object
Force                             object
Employed                          object
Unemployed                        object
(%)                               object
dtype: object

In [173]:
# Getting the data from Oct 2020
unemployment = unemployment[unemployment.Period == 'Oct-20 p']
unemployment = unemployment.reset_index()
unemployment.head()

,index,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
0,41848,CN0100100000000,1,1,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5
1,41849,CN0100300000000,1,3,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0
2,41850,CN0100500000000,1,5,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8
3,41851,CN0100700000000,1,7,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8
4,41852,CN0100900000000,1,9,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3


In [174]:
test = unemployment[unemployment['County Name/State Abbreviation'] == 'Presque Isle County, MI']
test.head()

,index,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
1300,43148,CN2614100000000,26,141,"Presque Isle County, MI",Oct-20 p,"4,829","4,602",227,4.7


In [175]:
# Creating a Fips column
unemployment['Fips'] = 0
for i in range(0,len(unemployment)):

    if(unemployment['Code'][i]) < 10:
        if (unemployment['Code.1'][i]) < 10:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
        elif (unemployment['Code.1'][i]) < 100:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
        else:
            unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '' + str(unemployment['Code.1'][i])
    elif (unemployment['Code.1'][i]) < 10:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
    elif (unemployment['Code.1'][i]) < 100:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
    else:
        unemployment['Fips'][i] = str(unemployment['Code'][i]) + '' + str(unemployment['Code.1'][i])
        
unemployment.head()

<ipython-input-175-dbd994a91222>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '00' + str(unemployment['Code.1'][i])
<ipython-input-175-dbd994a91222>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unemployment['Fips'][i] = '0'+str(unemployment['Code'][i]) + '0' + str(unemployment['Code.1'][i])
<ipython-input-175-dbd994a91222>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,index,LAUS Code,Code,Code.1,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,41848,CN0100100000000,1,1,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
1,41849,CN0100300000000,1,3,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0,1003
2,41850,CN0100500000000,1,5,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8,1005
3,41851,CN0100700000000,1,7,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8,1007
4,41852,CN0100900000000,1,9,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3,1009


In [176]:
# Dropping columns we don't need
unemployment = unemployment.drop(['LAUS Code','Code','Code.1', 'index'], axis = 1)
unemployment.head()

,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,"Autauga County, AL",Oct-20 p,"25,226","24,095","1,131",4.5,1001
1,"Baldwin County, AL",Oct-20 p,"95,660","90,920","4,740",5.0,1003
2,"Barbour County, AL",Oct-20 p,"9,634","8,880",754,7.8,1005
3,"Bibb County, AL",Oct-20 p,"8,624","8,127",497,5.8,1007
4,"Blount County, AL",Oct-20 p,"24,687","23,860",827,3.3,1009


In [177]:
poverty = pd.read_csv('data/acs_poverty.csv', header = 1)

In [178]:
poverty.head()

,id,Geographic Area Name,Estimate!!Total!!Population for whom poverty status is determined,Margin of Error!!Total!!Population for whom poverty status is determined,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Under 5 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!5 to 17 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Related children of householder under 18 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years!!Related children of householder under 18 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!18 to 34 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!18 to 34 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!35 to 64 years,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!18 to 64 years!!35 to 64 years,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!60 years and over,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!60 years and over,Estimate!!Total!!Population for whom poverty status is determined!!AGE!!65 years and over,Margin of Error!!Total!!Population for whom poverty status is determined!!AGE!!65 years and over,Estimate!!Total!!Population for whom poverty status is determined!!SEX!!Male,Margin of Error!!Total!!Population for whom poverty status is determined!!SEX!!Male,Estimate!!Total!!Population for whom poverty status is determined!!SEX!!Female,Margin of Error!!Total!!Population for whom poverty status is determined!!SEX!!Female,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Estimate!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Margin of Error!!Total!!Population for whom poverty status is determined!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Estimate!!Total!!Population for whom 

In [179]:
# Getting just the columns that we need and renaming

poverty = poverty[['id','Geographic Area Name','Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED']]
poverty = poverty.rename(columns = {'Estimate!!Percent below poverty level!!UNRELATED INDIVIDUALS FOR WHOM POVERTY STATUS IS DETERMINED':'Poverty Percent, All Ages'})
poverty.head()

,id,Geographic Area Name,"Poverty Percent, All Ages"
0,0500000US04001,"Apache County, Arizona",42.8
1,0500000US04003,"Cochise County, Arizona",27.6
2,0500000US04005,"Coconino County, Arizona",38.0
3,0500000US04013,"Maricopa County, Arizona",21.0
4,0500000US04015,"Mohave County, Arizona",29.3


In [180]:
#Merging poverty into our 2020 data set
df2020 = pd.merge(left = df2020, right = poverty, on = 'id')
df2020.head()

,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages"
0,0500000US04001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8
1,0500000US04003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6
2,0500000US04005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0
3,0500000US04013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0
4,0500000US04015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427,212181.0,0.889599,0.007051,0.009063,"Mohave County, Arizona",29.3


In [182]:
unemployment.dtypes

County Name/State Abbreviation    object
Period                            object
Force                             object
Employed                          object
Unemployed                        object
(%)                               object
Fips                               int64
dtype: object

In [183]:
# Merge in our unemployment data
# make the id column of our df2020 match the fips column we created in
# the unemployment table
unemployment['Fips'] = unemployment['Fips'].apply(str)
df2020["id"] = df2020["id"].str.replace("0500000US0", "")
df2020 = pd.merge(left = df2020, right = unemployment, left_on = 'id', right_on = 'Fips', how = 'left')
df2020.head()

,id,Geographic Area Name_x,Less than High School,At Least High School,At Least Bachelor's Degree,Graduate Degree,Estimate!!Total:,White (Not Latino) Population,African American Population,Asian American Population,Geographic Area Name,"Poverty Percent, All Ages",County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%),Fips
0,4001,"Apache County, Arizona",0.179969,0.314415,0.056851,0.053024,71887.0,0.210177,0.010030,0.002226,"Apache County, Arizona",42.8,"Apache County, AZ",Oct-20 p,"20,064","17,523","2,541",12.7,4001
1,4003,"Cochise County, Arizona",0.123738,0.271272,0.135565,0.089437,125922.0,0.859079,0.048943,0.022196,"Cochise County, Arizona",27.6,"Cochise County, AZ",Oct-20 p,"52,183","48,756","3,427",6.6,4003
2,4005,"Coconino County, Arizona",0.058941,0.224030,0.178253,0.145664,143476.0,0.638776,0.009507,0.019028,"Coconino County, Arizona",38.0,"Coconino County, AZ",Oct-20 p,"70,110","63,315","6,795",9.7,4005
3,4013,"Maricopa County, Arizona",0.121441,0.239276,0.201831,0.122930,4485414.0,0.790820,0.059332,0.042873,"Maricopa County, Arizona",21.0,"Maricopa County, AZ",Oct-20 p,"2,356,405","2,179,116","177,289",7.5,4013
4,4015,"Mohave County, Arizona",0.136021,0.347913,0.081868,0.048427,212181.0,0.889599,0.007051,0.009063,"Mohave County, Arizona",29.3,"Mohave County, AZ",Oct-20 p,"87,420","79,504","7,916",9.1,4015
